In [6]:
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import util
from model_detection import Model
from model_detection import kalman_filter
from model_detection import pnp_detection
from model_detection import robust_matcher
from model_detection import Mesh
import time

In [8]:
def kalman_filter_params(n_states, n_measurements, n_inputs, dt):
    """
    returns predefines kalman filter parameters
    namely, measurement matrix and transition matrix

    params:
    n_states: Number of states
    n_measurements: number of measurements
    n_inputs: number of control actions
    dt: time between measurements

    """
    pass

In [14]:
print("Started....")

video_source = "./data/test/box.mp4"
model_path = "./data/test/cookies_ORB.yml"
mesh_path = "./data/test/box.ply"
# load the model
print("Parsing and registering model/mesh....")

model = Model.loadModel(model_path)
keypoints_model = model.getKeypoints()
descriptors_model = model.getDescriptors()
model_3d_points = model.get3DPoints()
# load mesh
mesh = Mesh.loadMesh(mesh_path)

print("Model/Mesh registration is done")

# intrinsic camera parameters
# fx, fy, cx, cy
camera_params = util.load_camera_parameters("data/calib.npy")

# init kalman filter
n_states = 18 # the number of states
n_measurements = 6 # the number of measured states
n_inputs = 0 # the number of control actions
dt = 0.125  #time between measurements (1/FPS)
kf = kalman_filter( params=kalman_filter_params( n_states, n_measurements, n_inputs, dt ) )

# init pnp_detection
f = 55
sx, sy = 22.3, 14.9
width, height = 640, 480

pnp = pnp_detection( width*f/sx, height*f/sy, width/2, height/2, method="iterative")
# est pnp for kalman filter
pnp_est = pnp_detection(width*f/sx, height*f/sy, width/2, height/2)

# ransac parameters
ransac_confidence = 0.99
ransac_iterations = 500
max_reprojection_error = 6.0 # maximum allowed distance for inlier

# kalman parameters
kalman_min_inliers = 30

Started....
Parsing and registering model/mesh....
Model/Mesh registration is done


In [49]:
# initalize matcher
ratio_test = 0.7 # some default value
matcher = robust_matcher( ratio_test=ratio_test, feature_detector="ORB", matcher="FLANN", use_cross_check=False )

### main loop

In [50]:
# frame loop
frame_number = 0

cap = cv.VideoCapture(video_source)
# for online steaming
#cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera/no video presented.")
    exit()

while True:
    start_time = time.time()
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # step 1 - match the points between the model and the frame
    matches, kp_frame = matcher.fastMatch(frame, keypoints_model, descriptors_model)
    
    print("matches number", len(matches))

    # step 2 - 3d-2d correspondencies
    points_2d_matches, points_3d_matches = [], []
    for i in range(len(matches)):
        points_2d_matches.append(kp_frame[ matches[i].queryIdx ].pt)
        points_3d_matches.append( model_3d_points[matches[i].trainIdx] )
    # cast to numpy array
    points_2d_matches = np.array(points_2d_matches)
    points_3d_matches = np.array(points_3d_matches)

    # draw outliers
    util.drawPoints( frame, points_2d_matches, color="red")
    
    #cv.imshow('image window', frame)
    # add wait key. window waits until user presses a key
    #cv.waitKey(0)
    # and finally destroy/close all open windows
    #cv.destroyAllWindows()
    
    print("matches number", len(matches))
    # at least 4 matches are required for ransac estimation
    if len(matches) >= 4:
        # step 3 - estimate pose of the camera
        inliers = pnp.estimatePoseRansac(  points_3d_matches, points_2d_matches, \
            confidence=ransac_confidence,  iterations_count=ransac_iterations, 
            max_reporjection_error=max_reprojection_error )
    
        print("inliers found",len(inliers),"shape",inliers.shape, inliers[1])

        inlier_2d_points = points_2d_matches[ inliers.flatten() ]
        # draw the inliers
        util.drawPoints( frame, inlier_2d_points, color="green" )
        
        cv.imshow('image window', frame)
        # add wait key. window waits until user presses a key
        cv.waitKey(0)
        # and finally destroy/close all open windows
        cv.destroyAllWindows()

        # step 5
        # kalman filter 
        if len(inliers) >= kalman_min_inliers:

            R, t = pnp.getRotationTranslation()
            # update measurements, according to R and t
            measurements = kf.updateMeasurements(R,t)

        # estimate R and t from updated kalman filter
        estimated_R, estimated_t = kf.estimate()

        # step 6 - set estimated projection matrix
        pnp_est.setProjectionMatrix(estimated_R, estimated_t)

matches number 54
matches number 54
[[ 3.58244289e-01 -9.30796135e-01  7.26607442e-02  2.13354267e+01]
 [-5.21509112e-01 -2.64055237e-01 -8.11358785e-01  7.93126062e-01]
 [ 7.74396071e-01  2.52771411e-01 -5.80014947e-01  1.51115664e+02]]
inliers found 52 shape (52, 1) [1]


TypeError: cannot unpack non-iterable NoneType object